## Fine-Tuning Llama3 8B for Text Summarization üìÑ

ÎÖºÎ¨∏ SummarizationÏö© Îç∞Ïù¥ÌÑ∞ÏÖãÏúºÎ°ú Ïã§ÌóòÌï¥Î¥§ÏúºÎÇò, ÏßàÏù¥ ÎÑàÎ¨¥ Ï¢ãÏßÄ ÏïäÏùÄ Í¥ÄÍ≥ÑÎ°ú ÏßàÏù¥ Ï¢ãÏùÄ QA Îç∞Ïù¥ÌÑ∞ÏÖãÏùÑ ÌôúÏö©ÌïòÍ≥†Ïûê Ìï©ÎãàÎã§~  QÎ•º Í∑∏ÎÉ• ÎÇ¥Ïö©ÏùÑ Í∞ÑÎã®ÌïòÍ≤å ÏöîÏïΩÌï¥Îã¨ÎùºÍ≥† ÎÑ£ÏúºÎ©¥ ÎêòÍ≤†Ï£†?

#### Install & Import Dependencies

In [1]:
%%capture
# Unsloth, Xformers (Flash Attention) Î∞è Í∏∞ÌÉÄ ÌïÑÏöîÌïú Ìå®ÌÇ§ÏßÄÎì§ÏùÑ ÏÑ§ÏπòÌï¥Ïöî!
!pip install "unsloth[colab-new] @ git+https://github.com/unslothai/unsloth.git"
!pip install --no-deps "xformers<0.0.26" trl peft accelerate bitsandbytes

In [2]:
from unsloth import FastLanguageModel
import torch
from datasets import load_dataset

#### Load Model

In [3]:
# ÏµúÎåÄ ÏãúÌÄÄÏä§ Í∏∏Ïù¥ ÏÑ§Ï†ï. ÎÇ¥Î∂ÄÏ†ÅÏúºÎ°ú RoPE Ïä§ÏºÄÏùºÎßÅÏùÑ ÏûêÎèôÏúºÎ°ú ÏßÄÏõêÌï¥Ïöî!
max_seq_length = 2048 # ÏõêÌïòÎäî Í∞íÏúºÎ°ú ÏÑ§Ï†ï~

# Îç∞Ïù¥ÌÑ∞ ÌÉÄÏûÖ ÏÑ§Ï†ï. ÏûêÎèô Í∞êÏßÄÎ•º ÏúÑÌï¥ None, Tesla T4ÎÇò V100ÏùÄ Float16, Ampere Ïù¥ÏÉÅÏóêÏÑúÎäî Bfloat16 ÏÇ¨Ïö©ÌïòÏÑ∏Ïöî!
dtype = None

# Î©îÎ™®Î¶¨ ÏÇ¨Ïö©ÎüâÏùÑ Ï§ÑÏù¥Í∏∞ ÏúÑÌï¥ 4ÎπÑÌä∏ ÏñëÏûêÌôî ÏÇ¨Ïö©. ÌïÑÏöîÌïòÏßÄ ÏïäÎã§Î©¥ FalseÎ°ú ÏÑ§Ï†ï.
load_in_4bit = True

# 4bit pre quantized modelsÎ°ú 4x faster downloading + no OOMs.
fourbit_models = [
    "unsloth/mistral-7b-bnb-4bit",
    "unsloth/mistral-7b-instruct-v0.2-bnb-4bit",
    "unsloth/llama-2-7b-bnb-4bit",
    "unsloth/gemma-7b-bnb-4bit",
    "unsloth/gemma-7b-it-bnb-4bit", # Instruct version of Gemma 7b
    "unsloth/gemma-2b-bnb-4bit",
    "unsloth/gemma-2b-it-bnb-4bit", # Instruct version of Gemma 2b
    "unsloth/llama-3-8b-bnb-4bit", # [NEW] 15 Trillion token Llama-3
] # https://huggingface.co/unsloth Ïóê Îçî ÎßéÏùÄ Î™®Îç∏Ïù¥ ÏûàÏñ¥Ïöî~

# pretrained modelÍ≥º tokenizerÎ•º Î∂àÎü¨ÏòµÎãàÎã§
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/llama-3-8b-bnb-4bit", # ÏÇ¨Ïö©Ìï† Î™®Îç∏Ïùò Ïù¥Î¶Ñ
    max_seq_length = max_seq_length, # ÏµúÎåÄ ÏãúÌÄÄÏä§ Í∏∏Ïù¥
    dtype = dtype, # Îç∞Ïù¥ÌÑ∞ ÌÉÄÏûÖ
    load_in_4bit = load_in_4bit, # 4ÎπÑÌä∏ ÏñëÏûêÌôî ÏÇ¨Ïö© Ïó¨Î∂Ä
)


/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


config.json:   0%|          | 0.00/1.18k [00:00<?, ?B/s]

==((====))==  Unsloth: Fast Llama patching release 2024.5
   \\   /|    GPU: Tesla T4. Max memory: 14.748 GB. Platform = Linux.
O^O/ \_/ \    Pytorch: 2.2.1+cu121. CUDA = 7.5. CUDA Toolkit = 12.1.
\        /    Bfloat16 = FALSE. Xformers = 0.0.25.post1. FA = False.
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth


/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
Unused kwargs: ['_load_in_4bit', '_load_in_8bit', 'quant_method']. These kwargs are not used in <class 'transformers.utils.quantization_config.BitsAndBytesConfig'>.
/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


model.safetensors:   0%|          | 0.00/5.70G [00:00<?, ?B/s]

...

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/464 [00:00<?, ?B/s]

Unsloth: unsloth/llama-3-8b-bnb-4bit has no tokenizer.model file.
Just informing you about this - this is not a critical error.


Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


LoRA AdapterÎ•º Í∞ÄÏ†∏ÏòµÎãàÎã§~ Ïù¥Î†áÍ≤å ÌïòÎ©¥ Ï†ÑÏ≤¥ ÌååÎùºÎØ∏ÌÑ∞Ïùò 1~10%Îßå ÏóÖÎç∞Ïù¥Ìä∏ÌïòÏó¨ Îπ†Î•¥Í≥† Ìö®Ïú®Ï†ÅÏù∏ ÌïôÏäµÏù¥ Í∞ÄÎä•Ìï©ÎãàÎã§ üòÄ

In [4]:
model = FastLanguageModel.get_peft_model(
    model,
    r = 16, # Choose any number > 0 ! Suggested 8, 16, 32, 64, 128
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",],
    lora_alpha = 16,
    lora_dropout = 0, # Supports any, but = 0 is optimized
    bias = "none",    # Supports any, but = "none" is optimized
    # [NEW] "unsloth" uses 30% less VRAM, fits 2x larger batch sizes!
    use_gradient_checkpointing = "unsloth", # True or "unsloth" for very long context
    random_state = 3407,
    use_rslora = False,  # We support rank stabilized LoRA
    loftq_config = None, # And LoftQ
)

Unsloth 2024.5 patched 32 layers with 32 QKV layers, 32 O layers and 32 MLP layers.


#### Data Prep

In [5]:
alpaca_prompt = """Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
{}

### Input:
{}

### Response:
{}"""

In [6]:
EOS_TOKEN = tokenizer.eos_token # Must add EOS_TOKEN
# Remember to add the EOS_TOKEN to the tokenized output!! Otherwise you'll get infinite generations!

def formatting_prompts_func(examples):
    instructions = examples["question"]
    inputs       = examples["context"]
    outputs      = examples["answer"]
    texts = []
    for instruction, input, output in zip(instructions, inputs, outputs):
        text = alpaca_prompt.format(instruction, input, output) + EOS_TOKEN
        texts.append(text)
    return { "text" : texts, }

# Îç∞Ïù¥ÌÑ∞ÏÖã Î°úÎìú
dataset = load_dataset("hanyueshf/ml-arxiv-papers-qa", split="train")

# Îç∞Ïù¥ÌÑ∞ÏÖã Î≥ÄÌôò
dataset = dataset.map(formatting_prompts_func, batched=True)


Generating train split:   0%|          | 0/43713 [00:00<?, ? examples/s]

Map:   0%|          | 0/43713 [00:00<?, ? examples/s]

#### Train Model

Huggingface TRLÏóêÏÑú Ï†úÍ≥µÌïòÎäî SFTTrainerÎ•º Ïù¥Ïö©Ìï¥ÏÑú Î™®Îç∏ÏùÑ fine-tuning Ìï¥Î¥êÏöî~

SFTTrainerÎäî Supervised Fine-TuningÏúºÎ°ú RLHFÏóêÏÑú ÌïÑÏàòÏ†ÅÏù∏ Í≥ºÏ†ïÏûÖÎãàÎã§! [ÎßÅÌÅ¨](https://huggingface.co/docs/trl/sft_trainer)Ïóê Îì§Ïñ¥Í∞ÄÏÑú ÏßÅÏ†ë ÌôïÏù∏Ìï¥Î≥¥ÏÑ∏Ïöî üòé

In [7]:
from trl import SFTTrainer
from transformers import TrainingArguments

trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = dataset,
    dataset_text_field = "text",
    max_seq_length = max_seq_length,
    dataset_num_proc = 2,
    packing = False, # Can make training 5x faster for short sequences.
    args = TrainingArguments(
        per_device_train_batch_size = 2,
        gradient_accumulation_steps = 4,
        warmup_steps = 5,
        max_steps = 60,
        learning_rate = 2e-4,
        fp16 = not torch.cuda.is_bf16_supported(),
        bf16 = torch.cuda.is_bf16_supported(),
        logging_steps = 1,
        optim = "adamw_8bit",
        weight_decay = 0.01,
        lr_scheduler_type = "linear",
        seed = 3407,
        output_dir = "outputs",
    ),
)

/usr/local/lib/python3.10/dist-packages/multiprocess/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()


Map (num_proc=2):   0%|          | 0/43713 [00:00<?, ? examples/s]

/usr/local/lib/python3.10/dist-packages/multiprocess/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()
max_steps is given, it will override any value given in num_train_epochs


In [8]:
trainer_stats = trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs = 1
   \\   /|    Num examples = 43,713 | Num Epochs = 1
O^O/ \_/ \    Batch size per device = 2 | Gradient Accumulation steps = 4
\        /    Total batch size = 8 | Total steps = 60
 "-____-"     Number of trainable parameters = 41,943,040


#### Inference

Inference Ìï¥Î¥ÖÏãúÎã§~ outputÏùÑ ÎπÑÏõåÎë¨ÏÑú Î™®Îç∏Ïù¥ ÏßÅÏ†ë ÏÉùÏÑ±Ìï† Ïàò ÏûàÎèÑÎ°ù Ìï©ÎãàÎã§ ü§ì

instructionÏùÄ ÏöîÏïΩÌï¥Îã¨ÎùºÍ≥† ÎÑ£Í≥†, inputÏùÄ llama2 ÎÖºÎ¨∏ Ï†úÎ™©Í≥º abstractÎ•º Ìï©Ï≥êÏÑú ÎÑ£ÏóàÏñ¥Ïöî~

In [11]:

FastLanguageModel.for_inference(model) # Enable native 2x faster inference
inputs = tokenizer(
[
    alpaca_prompt.format(
        "Summarize what the paper is about", # instruction
        "Federated learning: Applications, challenges and future directions.Federated learning (FL) is a system in which a central aggregator coordinates the efforts of multiple clients to solve machine learning problems. This setting allows training data to be dispersed in order to protect privacy. The purpose of this paper is to provide an overview of FL systems with a focus on healthcare. FL is evaluated here based on its frameworks, architectures, and applications. It is shown here that FL solves the preceding issues with a shared global deep learning (DL) model via a central aggregator server. This paper examines recent developments and provides a comprehensive list of unresolved issues, inspired by the rapid growth of FL research. In the context of FL, several privacy methods are described, including secure multiparty computation, homomorphic encryption, differential privacy, and stochastic gradient descent. Furthermore, a review of various FL classes, such as horizontal and vertical FL and federated transfer learning, is provided. FL has applications in wireless communication, service recommendation, intelligent medical diagnosis systems, and healthcare, all of which are discussed in this paper. We also present a thorough review of existing FL challenges, such as privacy protection,communication cost, system heterogeneity, and unreliable model upload, followed by future research directions.", # input
        "", # output - leave this blank for generation!
    )
], return_tensors = "pt").to("cuda")

outputs = model.generate(**inputs, max_new_tokens = 64, use_cache = True)
tokenizer.batch_decode(outputs)

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


['<|begin_of_text|>Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.\n\n### Instruction:\nSummarize what the paper is about\n\n### Input:\nFederated learning: Applications, challenges and future directions.Federated learning (FL) is a system in which a central aggregator coordinates the efforts of multiple clients to solve machine learning problems. This setting allows training data to be dispersed in order to protect privacy. The purpose of this paper is to provide an overview of FL systems with a focus on healthcare. FL is evaluated here based on its frameworks, architectures, and applications. It is shown here that FL solves the preceding issues with a shared global deep learning (DL) model via a central aggregator server. This paper examines recent developments and provides a comprehensive list of unresolved issues, inspired by the rapid growth of FL research. In the context 

In [13]:
# Rogue Score Calculation
!pip install rouge
from rouge import Rouge

input = "Federated learning: Applications, challenges and future directions.Federated learning (FL) is a system in which a central aggregator coordinates the efforts of multiple clients to solve machine learning problems. This setting allows training data to be dispersed in order to protect privacy. The purpose of this paper is to provide an overview of FL systems with a focus on healthcare. FL is evaluated here based on its frameworks, architectures, and applications. It is shown here that FL solves the preceding issues with a shared global deep learning (DL) model via a central aggregator server. This paper examines recent developments and provides a comprehensive list of unresolved issues, inspired by the rapid growth of FL research. In the context of FL, several privacy methods are described, including secure multiparty computation, homomorphic encryption, differential privacy, and stochastic gradient descent. Furthermore, a review of various FL classes, such as horizontal and vertical FL and federated transfer learning, is provided. FL has applications in wireless communication, service recommendation, intelligent medical diagnosis systems, and healthcare, all of which are discussed in this paper. We also present a thorough review of existing FL challenges, such as privacy protection,communication cost, system heterogeneity, and unreliable model upload, followed by future research directions."
reference = "Federated Learning (FL) is a system that allows multiple clients to work together to train machine learning models without having to share data. The technology coordinates processes by a central server while protecting privacy. This paper covers the overview, framework, architecture, application field, privacy protection method, type, application field, challenges, and future research directions of FL, with a particular focus on the healthcare field."
output = "The paper provides an overview of Federated Learning (FL) systems with a focus on healthcare. It discusses FL frameworks, architectures, and applications, evaluating its effectiveness in solving issues related to data privacy. The paper highlights recent developments in FL and outlines unresolved issues, inspired by the rapid growth of research in this area."

rouge = Rouge()
ref_score = rouge.get_scores(input, reference)
output_score = rouge.get_scores(input, output)

print(ref_score)
print(output_score)

[{'rouge-1': {'r': 0.5892857142857143, 'p': 0.2426470588235294, 'f': 0.34374999586805555}, 'rouge-2': {'r': 0.22580645161290322, 'p': 0.07142857142857142, 'f': 0.10852712813172298}, 'rouge-l': {'r': 0.5357142857142857, 'p': 0.22058823529411764, 'f': 0.3124999958680556}}]
[{'rouge-1': {'r': 0.782608695652174, 'p': 0.2647058823529412, 'f': 0.3956043918270741}, 'rouge-2': {'r': 0.39215686274509803, 'p': 0.10204081632653061, 'f': 0.16194331656116318}, 'rouge-l': {'r': 0.7391304347826086, 'p': 0.25, 'f': 0.37362636984905206}}]


Ïñ¥Îñ§Í∞ÄÏöî? ÍΩ§ responseÍ∞Ä Í¥úÏ∞ÆÏ£†? üëç

#### Save & Load Fine-tuned Model

ÏµúÏ¢Ö Î™®Îç∏ÏùÑ LoRA adaptorÎ°ú Ï†ÄÏû•ÌïòÍ∏∞ ÏúÑÌï¥ huggingfaceÏùò push_to_hubÎ•º Ïù¥Ïö©ÌïòÍ±∞ÎÇò Î°úÏª¨ Ï†ÄÏû•ÏùÑ ÏúÑÌï¥ save_pretrainedÎ•º ÏÇ¨Ïö©ÌïòÎ©¥ Îê©ÎãàÎã§!

Îã§Îßå Ïù¥Îäî LoRA adaptorÎßå Ï†ÄÏû•Ìï† Îøê Ï†ÑÏ≤¥ Î™®Îç∏ÏùÄ Ï†ÄÏû•ÌïòÏßÄ ÏïäÏïÑÏöî!

In [14]:
model.save_pretrained("lora_model") # Local saving
tokenizer.save_pretrained("lora_model")
# model.push_to_hub("your_name/lora_model", token = "...") # Online saving
# tokenizer.push_to_hub("your_name/lora_model", token = "...") # Online saving

('lora_model/tokenizer_config.json',
 'lora_model/special_tokens_map.json',
 'lora_model/tokenizer.json')

Î∞©Í∏à inferenceÎ•º ÏúÑÌï¥ Ï†ÄÏû•Ìïú LoRA adapterÎ•º ÏÇ¨Ïö©ÌïòÍ≥† Ïã∂Îã§Î©¥, `False`Î•º `True`Î°ú ÏÑ∏ÌåÖÌïòÏÑ∏Ïöî!

In [15]:
if True:
    from unsloth import FastLanguageModel
    model, tokenizer = FastLanguageModel.from_pretrained(
        model_name = "lora_model",
        max_seq_length = max_seq_length,
        dtype = dtype,
        load_in_4bit = load_in_4bit,
    )
    FastLanguageModel.for_inference(model) # Enable native 2x faster inference

inputs = tokenizer(
[
    alpaca_prompt.format(
        "Summarize what the paper is about", # instruction
        "How Good Are Low-bit Quantized LLAMA3 Models? An Empirical Study. Meta‚Äôs LLAMA family has become one of the most powerful open-source Large Language Model (LLM) series. Notably, LLAMA3 models have recently been released and achieve impressive performance across various with super-large scale pre-training on over 15T tokens of data. Given the wide application of lowbit quantization for LLMs in resource-limited scenarios, we explore LLAMA3‚Äôs capabilities when quantized to low bit-width. This exploration holds the potential to unveil new insights and challenges for low-bit quantization of LLAMA3 and other forthcoming LLMs, especially in addressing performance degradation problems that suffer in LLM compression. Specifically, we evaluate the 10 existing posttraining quantization and LoRA-finetuning methods of LLAMA3 on 1-8 bits and diverse datasets to comprehensively reveal LLAMA3‚Äôs low-bit quantization performance. Our experiment results indicate that LLAMA3 still suffers nonnegligent degradation in these scenarios, especially in ultra-low bit-width. This highlights the significant performance gap under low bit-width that needs to be bridged in future developments. We expect that this empirical study will prove valuable in advancing future models, pushing the LLMs to lower bit-width with higher accuracy for being practical. ", # input
        "", # output - leave this blank for generation!
    )
], return_tensors = "pt").to("cuda")

outputs = model.generate(**inputs, max_new_tokens = 64, use_cache = True)
tokenizer.batch_decode(outputs)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


==((====))==  Unsloth: Fast Llama patching release 2024.5
   \\   /|    GPU: Tesla T4. Max memory: 14.748 GB. Platform = Linux.
O^O/ \_/ \    Pytorch: 2.2.1+cu121. CUDA = 7.5. CUDA Toolkit = 12.1.
\        /    Bfloat16 = FALSE. Xformers = 0.0.25.post1. FA = False.
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth


Unused kwargs: ['_load_in_4bit', '_load_in_8bit', 'quant_method']. These kwargs are not used in <class 'transformers.utils.quantization_config.BitsAndBytesConfig'>.
/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


Unsloth: lora_model has no tokenizer.model file.
Just informing you about this - this is not a critical error.


Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


["<|begin_of_text|>Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.\n\n### Instruction:\nSummarize what the paper is about\n\n### Input:\nHow Good Are Low-bit Quantized LLAMA3 Models? An Empirical Study. Meta‚Äôs LLAMA family has become one of the most powerful open-source Large Language Model (LLM) series. Notably, LLAMA3 models have recently been released and achieve impressive performance across various with super-large scale pre-training on over 15T tokens of data. Given the wide application of lowbit quantization for LLMs in resource-limited scenarios, we explore LLAMA3‚Äôs capabilities when quantized to low bit-width. This exploration holds the potential to unveil new insights and challenges for low-bit quantization of LLAMA3 and other forthcoming LLMs, especially in addressing performance degradation problems that suffer in LLM compression. Specifically, we evaluate the 1

#### Í≥ºÏ†ú

1. ÏûêÏã†Ïùò ÏöîÏïΩÍ≥º Î™®Îç∏Ïùò ÏöîÏïΩÏùò Rouge scoreÎ•º ÎπÑÍµêÌï¥Ï£ºÏÑ∏Ïöî
2. Rouge Ï†êÏàòÏôÄ Ïã§Ï†ú Ï£ºÍ¥ÄÏ†ÅÏù∏ ÏöîÏïΩ ÏßàÏóê Ï∞®Ïù¥Í∞Ä ÏûàÎÇòÏöî? (ÎÇòÏùò ÏöîÏïΩÏù¥ Îçî Ï¢ãÏùÄ Í≤É Í∞ôÏùÄÎç∞ Rouge ÏßÄÌëúÎäî Îçî ÎÇÆÍ≤å ÎÇòÏôîÎã§ÎçòÏßÄ‚Ä¶?) Ïù¥Îü∞ Ï∞®Ïù¥Í∞Ä Î∞úÏÉùÌïòÎäî Ïù¥Ïú†Í∞Ä Î¨¥ÏóáÏù∏Í∞ÄÏöî?

3. Ïã§Ï†ú ÏöîÏïΩ ÏÑ±Îä•ÏùÑ Î≥¥Îã§ Ïûò Ï∏°Ï†ïÌïòÍ∏∞ ÏúÑÌï¥ÏÑú evaluation metricÏúºÎ°ú Î¨¥ÏóáÏù¥ Îçî Í≥†Î†§ÎêòÏñ¥ÏïºÌïòÎäîÏßÄ ÏûêÏú†Î°≠Í≤å ÏûëÏÑ±Ìï¥Ï£ºÏÑ∏Ïöî.



1. Ï†ÑÎ∞òÏ†ÅÏúºÎ°ú Î™®Îç∏ ÏöîÏïΩÎ¨∏Ïùò Rouge scoreÍ∞Ä Ï†ú ÏöîÏïΩÎ≥¥Îã§ ÎÜíÍ≤å ÎÇòÏôîÏäµÎãàÎã§. ÌäπÌûà RecallÏùÑ ÎπÑÍµêÌïòÎ©¥, Î™®Îç∏Ïùò ÏöîÏïΩÎ¨∏Ïù¥ ÏõêÎ¨∏Ïùò ÎÇ¥Ïö©ÏùÑ Îçî ÎßéÏù¥ Ìè¨Ìï®ÌïòÍ≥† ÏûàÏäµÎãàÎã§. Ïù¥Î•º ÌÜµÌï¥ Î™®Îç∏Ïù¥ ÏÉùÏÑ±Ìïú ÏöîÏïΩÎ¨∏Ïù¥ Ï†úÍ∞Ä ÏûëÏÑ±Ìïú ÏöîÏïΩÎ≥¥Îã§ ÏõêÎ¨∏Ïùò ÎÇ¥Ïö©ÏùÑ Îçî Ïûò Î∞òÏòÅÌïòÍ≥† ÏûàÎã§Îäî Í≤∞Í≥ºÎ•º ÎÇòÌÉÄÎÉÖÎãàÎã§.

2. RougeÎäî n-gramÏùÑ Í∏∞Î∞òÏúºÎ°ú ÌïòÎäî ÌèâÍ∞Ä ÏßÄÌëúÎ°ú, ÏõêÎ¨∏Ïùò ÏùòÎØ∏ÎÇò Î¨∏Îß•ÏùÑ Í≥†Î†§ÌïòÏßÄ ÏïäÍ≥† Îã®ÏàúÌûà Îã®Ïñ¥Ïùò Ï§ëÎ≥µÎèÑÎ•º Í∏∞Ï§ÄÏúºÎ°ú Ï†êÏàòÎ•º Ï∏°Ï†ïÌï©ÎãàÎã§. Îî∞ÎùºÏÑú Îã®ÏàúÌûà ÏõêÎ¨∏Í≥º ÎπÑÏä∑Ìïú Îã®Ïñ¥Î•º ÏÇ¨Ïö©ÌïòÎ©¥ Îçî ÎÜíÏùÄ Ï†êÏàòÎ•º Î∞õÏùÑ Ïàò ÏûàÏäµÎãàÎã§. Í∑∏Îü¨ÎÇò ÏÇ¨ÎûåÏù¥ ÌåêÎã®ÌïòÎäî ÏöîÏïΩÏùò ÏßàÏùÄ ÎÇ¥Ïö©Ïùò Ï†ïÌôïÏÑ±Ïù¥ÎÇò Í∞ÄÎèÖÏÑ± Îì± Îã§ÏñëÌïú Í¥ÄÏ†êÏùÑ Í∏∞Ï§ÄÏúºÎ°ú Ï¢ÖÌï©Ï†ÅÏúºÎ°ú Í≥†Î†§ÌïòÍ∏∞ ÎïåÎ¨∏Ïóê Ïù¥Îü∞ Ï∞®Ïù¥Í∞Ä Î∞úÏÉùÌï©ÎãàÎã§.


3. BERTScore: BERTÏ≤òÎüº ÏÇ¨Ï†Ñ ÌïôÏäµÎêú LLMÏùÑ ÏÇ¨Ïö©Ìï¥ ÏõêÎ¨∏Í≥º ÏöîÏïΩÎ¨∏Ïùò Ïú†ÏÇ¨ÎèÑÎ•º ÌåêÎã®ÌïòÏó¨ ÌèâÍ∞ÄÌï† Ïàò ÏûàÏäµÎãàÎã§. Ïù¥Îäî rougeÏôÄ Îã§Î•¥Í≤å Î¨∏Ïû• ÏàòÏ§ÄÏóêÏÑú Ïú†ÏÇ¨ÎèÑÎ•º ÌååÏïÖÌï† Ïàò ÏûàÏñ¥ Îçî Ïûò Ï∏°Ï†ïÌï† Ïàò ÏûàÏäµÎãàÎã§.